VGGT
====

**VGGT: Visual Geometry Grounded Transformer**

 * Paper: https://arxiv.org/abs/2503.11651

![VGG Overview](../assets/vggt_overview.png)

![VGGT Model](../assets/vggt_model_overview.png)

```bash
git clone https://github.com/facebookresearch/vggt.git

pip install torch torchvision
pip install transformers
pip install einops
```

In [ ]:
import sys
import torch
sys.path.append("vggt")  # remember to clone the repo
from vggt.models.vggt import VGGT
from vggt.utils.load_fn import load_and_preprocess_images

device = "cuda" if torch.cuda.is_available() else "cpu"
# bfloat16 is supported on Ampere GPUs (Compute Capability 8.0+) 
dtype = (
    torch.bfloat16
    if torch.cuda.get_device_capability()[0] >= 8
    else torch.float16
)
print(f"Using device: {device}, dtype: {dtype}")

model = VGGT.from_pretrained("facebook/VGGT-1B")
model.eval().to(device);

/home/pyml/anaconda3/envs/py311/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda, dtype: torch.bfloat16
